The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [3]:
import cProfile
import timeit

In [34]:
from mip import Model, MINIMIZE, MAXIMIZE, BINARY, xsum, OptimizationStatus
from itertools import product, combinations

In [5]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [6]:
wide = evaluator.evaluate_corners_wide().reset_index()

In [7]:
wide.head(26)

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.552447,2.066952,-0.003592,0.974196,-0.142710,188.094932,0.063096,10003.881725
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.472151,2.066167,-0.003592,0.975175,-0.140198,188.094879,0.063096,10003.883164
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.301388,2.066841,-0.003592,0.974380,-0.134854,188.094949,0.063096,10003.884565
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.475650,2.067201,-0.003592,0.962298,-0.136619,188.094947,0.063096,10067.675941
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.395354,2.066415,-0.003592,0.963277,-0.134107,188.094893,0.063096,10067.677381
5,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,5000000.0,-1.224591,2.067089,-0.003592,0.962483,-0.128764,188.094964,0.063096,10067.678782
6,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,-1.395378,2.067584,-0.003592,0.954555,-0.131997,188.094893,0.063096,10110.860175
7,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,1000000.0,-1.315081,2.066798,-0.003592,0.955534,-0.129485,188.094839,0.063096,10110.861615
8,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,5000000.0,-1.144319,2.067472,-0.003592,0.954739,-0.124142,188.094910,0.063096,10110.863015
9,0.0,0.0,0.0,0.0,0.0,2500000.0,0.0,0.0,-1.437190,2.078564,-0.003592,0.932174,-0.137945,188.094914,0.063096,10023.619936


In [8]:
categories = ['Polysilicon', 'Power Electronics', 'Soft Costs']

metric_obj = ['Efficiency']

metric_constraint = ['Hazardous']

Pull out the investment values and metric values from the data set.

In [9]:
# Investment levels = interval endpoints
inv_levels = wide.loc[:26,categories].values.tolist()

# Elicited metric values for objective function
m = wide.loc[:26,metric_obj].values.tolist()

# Elicited metric values for constraint
const = wide.loc[:26, metric_constraint].values.tolist()

# Number of endpoints
I = len(inv_levels)

Instantiate the MIP optimization problem.

In [107]:
example = Model(sense=MAXIMIZE)

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [108]:
lmbd_vars = []

for i in range(I):
    lmbd_vars += [example.add_var(name='lmbd_' + str(i), lb=0.0, ub=1.0)]

Create binary (integer) variables $y_{in_i}$ and constraints on $y$ and $\lambda$ that restrict optimal solutions to the intervals between endpoints (elicited data).

In [109]:
bin_vars = []
bin_count = 0

for i in range(I):
    for j in range(i, I):
        if j != i:
            # create binary variable
            bin_vars += [example.add_var(name='y_' + str(i) + str(j),
                                         var_type=BINARY)]
            # add binary/lambda variable constraint
            example += bin_vars[bin_count] <= lmbd_vars[i] + lmbd_vars[j], 'Interval_Constraint_' + str(i) + str(j)
            bin_count += 1

Define upper bound for total budget constraint and per-category upper bounds.

In [110]:
B = 3000000.0
max_amount = [3000000.0, 2300000.0, 3000000.0]
const_amount = 0.95

Create total budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [111]:
example += xsum(lmbd_vars[i] * inv_levels[i][j] 
                for i in range(I)
                for j in range(len(inv_levels[i]))) <= B, 'Total_Budget'

Create constraints for budget within each investment category.

In [112]:
for j in range(len(categories)):
    example += xsum(lmbd_vars[i] * [el[j] for el in inv_levels][i] 
                    for i in range(I)) <= max_amount[j], 'Budget_for_' + categories[j].replace(' ', '')

Create minimum-metric constraint on the lambda variables.

In [113]:
example += xsum(lmbd_vars[i] * const[i][0] for i in range(I)) <= const_amount

Create two sets of constraints:
* Convexity constraints on $\lambda$ variables: the $\lambda$s must sum to 1.
* Constrain binary $y$ variables within each category such that the decision variables lie within exactly one interval.

In [114]:
example += sum(lmbd_vars) == 1, 'Lambda_Sum'

The sum over all binary variables is constrained to be exactly 1 to restrict optimal solutions to the linear intervals between endpoints (elicited data).

In [115]:
example += sum(bin_vars) == 1, 'Binary_Sum'

Create objective function: Metric objective as a function of $\lambda$s and elicited data.

In [116]:
example.objective = xsum(m[i][0] * lmbd_vars[i] for i in range(I))

Save model to a `lp` file for manual examination (optional; un-comment to save).

In [20]:
#example.write('example.lp')

Optimize.

In [117]:
status = example.optimize()

In [118]:
status

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, if a feasible solution was found, or an estimate for the objective function bound if no feasible solution was found.

In [119]:
if status == OptimizationStatus.OPTIMAL:
    print('optimal objective function value: ', example.objective_value)
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('objective function bound (estimate): ', example.objective_bound)
else:
    print('no solution found: ', status)

optimal objective function value:  2.0948990359828987


Get the optimal $\lambda$ and $y$ values. Store as separate lists and also print out.

In [120]:
lmbd_opt = []
y_opt = []

for v in example.vars:
    if 'lmbd' in v.name:
        lmbd_opt += [v.x]
    elif 'y' in v.name:
        y_opt += [v.x]
    if v.x > 1e-6:
        print('{} : {}'.format(v.name, v.x))

lmbd_0 : 0.33333333333333337
lmbd_18 : 0.6666666666666666
y_018 : 1.0


Calculate the optimal investment levels.

In [121]:
inv_levels_opt = {}

for i in range(len(categories)):
    inv_levels_opt[categories[i]] = sum([lmbd_opt[j] * [el[i] for el in inv_levels][j] 
                                         for j in range(len(lmbd_opt))])

inv_levels_opt

{'Polysilicon': 3000000.0, 'Power Electronics': 0.0, 'Soft Costs': 0.0}

Calculate the optimal total investment.

In [122]:
total_inv_levels_opt = sum(inv_levels_opt.values())
total_inv_levels_opt

3000000.0